In [1]:
import sys
import os
sys.path.append(r"D:\CursorProj\Chem-Ontology-Constructor")
os.environ["PROJECT_ROOT"] = "D:\\CursorProj\\Chem-Ontology-Constructor\\"

In [2]:
from autonogy_constructor.assertions import chemonto_with_entities_assertions, chemonto_with_elements_assertions, chemonto_with_data_properties_assertions, chemonto_with_object_properties_assertions
from autonogy_constructor.ontology_merge import merge_ontology
from src.ontology.preprocess import create_metadata_properties


d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
import dspy

lm = dspy.LM('openai/gpt-4o', temperature=0)
dspy.configure(lm=lm)
teacher = dspy.LM("openai/gpt-4o",temperature=1)

In [35]:
def read_and_split_text(file_path, debug=True):
    with open(file_path, 'r', encoding='utf-8') as f:
        # 标准化所有换行符
        content = f.read().replace('\r\n', '\n')
    
    # 移除每行末尾的空白字符
    lines = [line.rstrip() for line in content.split('\n')]
    
    blocks = []
    current_block = []
    block_count = 0
    
    for i, line in enumerate(lines):
        if debug:
            print(f"Line {i}: '{line}' (Length: {len(line)})")
        
        if not line:  # 遇到空行就分块
            if current_block:
                blocks.append('\n'.join(current_block))
                block_count += 1
                if debug:
                    print(f"找到第 {block_count} 个分块点，在第 {i} 行")
            current_block = []
        else:
            current_block.append(line)
    
    # 添加最后一个块
    if current_block:
        blocks.append('\n'.join(current_block))
        block_count += 1
    
    if debug:
        print(f"\n总共找到 {block_count} 个块")
        lens = [len(block) for block in blocks]
        print(f"每个块的字符长度: {lens}")
        print(f"平均长度: {sum(lens) / len(lens)}")
    
    return blocks

In [36]:

data = read_and_split_text(r'D:\CursorProj\Chem-Ontology-Constructor\tests\Supramolecular\data\supramolecule\Angew-2020 - GuoDS-Biomedical Applications of Calixarenes.txt')


Line 0: 'Calixarenes' (Length: 11)
Line 1: 'Biomedical Applications of Calixarenes: State of the Art' (Length: 56)
Line 2: 'and Perspectives' (Length: 16)
Line 3: 'Abstract:' (Length: 9)
Line 4: 'Calixarenes (CAs), representing the third generation of supramolec-' (Length: 67)
Line 5: 'ular hosts and one of the most widely studied macrocyclic scaffolds,' (Length: 68)
Line 6: 'offer (almost) unlimited structure and application possibilities due to' (Length: 71)
Line 7: 'their ease of modification, which allows one to establish a large' (Length: 65)
Line 8: 'molecular library as a material basis for diverse biomedical applica-' (Length: 69)
Line 9: 'tions. Moreover, CAs and their derivatives engage in various nonco-' (Length: 67)
Line 10: 'valent interactions for the facile recognition of guests including bio-' (Length: 71)
Line 11: 'active molecules and are also important building blocks for the fab-' (Length: 68)
Line 12: 'rication of supramolecular architectures. In view of their mole

In [37]:
data = [{"content": text, "source": " https://doi.org/10.1002/anie.201916380"} for text in data]
data

[{'content': 'Calixarenes\nBiomedical Applications of Calixarenes: State of the Art\nand Perspectives\nAbstract:\nCalixarenes (CAs), representing the third generation of supramolec-\nular hosts and one of the most widely studied macrocyclic scaffolds,\noffer (almost) unlimited structure and application possibilities due to\ntheir ease of modification, which allows one to establish a large\nmolecular library as a material basis for diverse biomedical applica-\ntions. Moreover, CAs and their derivatives engage in various nonco-\nvalent interactions for the facile recognition of guests including bio-\nactive molecules and are also important building blocks for the fab-\nrication of supramolecular architectures. In view of their molecular\nrecognition and self-assembly properties, CAs are extensively applied\nin biosensing, bioimaging, and drug/gene delivery. Additionally, some\nCA derivatives exhibit biological activities and can therefore be used as\nnew therapeutic agents. Herein, we su

In [38]:
for index, chunk in enumerate(data):
# index = 4
# chunk = data[index]
    print(f"第{index+1}个片段")
    test_context = chunk["content"]
    source = chunk["source"]
    print(f"前30个字符: {test_context[:30]}")
    temp = chemonto_with_entities_assertions(test_context)
    elements = chemonto_with_elements_assertions(test_context, temp)
    data_properties =   chemonto_with_data_properties_assertions(test_context, temp)
    object_properties = chemonto_with_object_properties_assertions(test_context, temp)
    create_metadata_properties()
    merge_ontology(
        temp.ontology_entities, 
        elements.ontology_elements, 
        data_properties.ontology_data_properties, 
        object_properties.ontology_object_properties,
        source
    )

2024/12/07 15:01:07 INFO dspy.primitives.assertions: SuggestionFailed: The ontology is missing a point because it does not explicitly state the hierarchical relationship between 'therapeutic_agent' and 'calixarenes' as a subclass of 'biomedical_applications'. To improve, the ontology should include this relationship to fully capture the text's implications.
2024/12/07 15:01:07 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not describe symmetric aspects of the same chemical scope, so this criterion is not applicable. To improve, ensure that if symmetric aspects exist, their instances are aligned in granularity.
2024/12/07 15:01:07 INFO dspy.primitives.assertions: SuggestionFailed: The ontology does not describe symmetric aspects of the same chemical scope, so this criterion is not applicable. To improve, ensure that if symmetric aspects exist, their instances are aligned in granularity.
2024/12/07 15:01:07 INFO dspy.primitives.assertions: SuggestionFailed: The ont

第1个片段
前30个字符: Calixarenes
Biomedical Applica
entities
Entities:
  - Name: calixarenes(CAs)
    Information: Calixarenes are the third generation of supramolecular hosts and widely studied macrocyclic scaffolds, offering extensive structure and application possibilities due to their ease of modification.
  - Name: supramolecular_architecture
    Information: Supramolecular architectures are structures formed by the self-assembly of molecules, including calixarenes, through noncovalent interactions.
  - Name: molecular_recognition
    Information: Molecular recognition refers to the ability of calixarenes to engage in noncovalent interactions for the recognition of guest molecules, including bioactive molecules.
  - Name: self-assembly_properties
    Information: Self-assembly properties of calixarenes allow them to form organized structures without external guidance, crucial for their application in supramolecular architectures.
  - Name: biosensing
    Information: Biosensing is an app

KeyboardInterrupt: 

# Improvement

1. 数据属性与对象属性混淆
2. 父类关系归属，是组成部分不能归属为父子类关系
3. 类名称不能有/，#
4. 复合实体需要拆分为多个独立的类
5. 

In [8]:
from config.settings import ONTOLOGY_CONFIG
onto = ONTOLOGY_CONFIG["ontology"]
list(onto.disjoint_classes())   


[]

In [3]:
from owlready2 import *
import owlready2

In [18]:
meta = ONTOLOGY_CONFIG["meta"]
meta_annotations = list(onto.annotation_properties())


In [19]:
default_world.full_text_search_properties.extend(meta_annotations)


In [20]:
default_world.full_text_search_properties

[rdf-schema.label, meta.source, meta.information, meta.hierarchy_information]

In [34]:
res = default_world.search(information = FTS("cryptands"), _bm25 = True)
# print(len(res))
# print(type(res[0][0]))

import pandas as pd

# 将搜索结果转换为DataFrame
df = pd.DataFrame(res, columns=['实体', '相关度分数'])
df


,实体,相关度分数
0,classes.pseudospherical_inner_cavity,-5.244072
1,classes.pyridine_nitrogen_atom,-5.056321
2,classes.naphthyridine_nitrogen_atom,-5.056321
3,classes.oligopyrrolic_cryptand_cage,-5.056321
4,classes.diformyl_pyrrole_precursor,-4.718457
5,classes.cryptate,-4.718457
6,classes.Schiff-base_condensation,-4.565910
7,classes.single-crystal_X-ray_structure,-4.565910
8,classes.molecular_design,-4.565910
9,data_properties.application_area,-4.422917


In [36]:
test = df["实体"][4]
test.information


['Chemical precursors used in the formation of oligopyrrolic cryptands.']

# 推理

In [4]:
from config.settings import ONTOLOGY_CONFIG
onto = ONTOLOGY_CONFIG["ontology"]


In [5]:

owlready2.JAVA_EXE = r"C:\Program Files\Java\jdk-23\bin\java.exe"


In [6]:
close_world(onto)

In [7]:
with onto:
    sync_reasoner(infer_property_values = True)

* Owlready2 * Running HermiT...
    C:\Program Files\Java\jdk-23\bin\java.exe -Xmx2000M -cp d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\hermit;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/SYK/AppData/Local/Temp/tmpy00cw0fl -Y
* Owlready2 * HermiT took 5.924745082855225 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [12]:
with onto:
    sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True, debug = 3)


* Owlready2 * Running Pellet...
    C:\Program Files\Java\jdk-23\bin\java.exe -Xmx2000M -cp d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\antlr-3.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;d:\AnacondaEnPs\envs\OntologyConstruction\Lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;d:\AnacondaEnPs\envs\OntologyCon

In [20]:
list(default_world.inconsistent_classes())

[classes.monometallic_CC,
 classes.coordination_cage(CC),
 classes.super_aryl-extended_calix(4)pyrrole(SAE-C(4)P),
 classes.acetonitrile,
 owl.Nothing,
 classes.reporter-spacer-receptor,
 classes.electrochemical_anion_sensor]

In [21]:
classes = ONTOLOGY_CONFIG["classes"]
acetonitrile = classes["acetonitrile"]
print(acetonitrile)

classes.acetonitrile


In [22]:
op = ONTOLOGY_CONFIG["object_properties"]
cocap = op["is_coencapsulated_with"]
cocap.information


['Solvent molecules like acetonitrile and water are coencapsulated with guest molecules in the coordination complex.']

In [23]:
acetonitrile.is_coencapsulated_with



[classes.guest_molecule(29),
 classes.guest_molecule(5a),
 classes.guest_molecule(23a)]

In [51]:
from owlready2 import *
from owlready2.class_construct import Restriction

def find_property_restrictions_from_is_a(cls, property_name):
    restrictions = []
    
    for r in cls.is_a:
        # 检查是否是Restriction类型
        if isinstance(r, Restriction):
            # 获取属性名称并比较
            if r.property.name == property_name:
                # 通过 type 属性来判断限制类型
                restrictions.append(r)
    
    return restrictions

def print_restrictions(cls, property_name):
    restrictions = find_property_restrictions_from_is_a(cls, property_name)
    
    if not restrictions:
        print(f"没有找到属性 {property_name} 的限制")
        return
        
    print(f"\n{cls.name} 类在属性 {property_name} 上的限制：")
    values = []
    for r in restrictions:
        if isinstance(r, Restriction):
            # 使用 type 属性来判断限制类型
            if r.type == ONLY:
                restriction_type = "ONLY"
                print(f"类型: {restriction_type}")
                print(f"值: {r.value}")
                print("---")
                values.append(r.value)
            elif r.type == SOME:
                restriction_type = "SOME"
                print(f"类型: {restriction_type}")
                print(f"值: {r.value}")
                print("---")
                values.append(r.value)
    return tuple(values)
# 使用示例
my_class = classes["acetonitrile"]
property_name = "is_coencapsulated_with"
values = print_restrictions(my_class, property_name)
infos = []
for value in values + (my_class,):
    if isinstance(value, owlready2.entity.ThingClass):
        info = {"name": value.name, "information": value.information, "source": value.source}
        infos.append(info)
infos_str = ""
for info in infos:
    infos_str += f"\nInformation of '{info['name']}':\n"
    for item in info['information']:
        infos_str += f"  - {item}\n"
    infos_str += f"Source of '{info['name']}':\n"
    for item in info['source']:
        infos_str += f"  - {item}\n"
print(infos_str)



acetonitrile 类在属性 is_coencapsulated_with 上的限制：
类型: SOME
值: classes.guest_molecule(23a)
---
类型: SOME
值: classes.guest_molecule(29)
---
类型: SOME
值: classes.guest_molecule(5a)
---
类型: ONLY
值: classes.acid
---
类型: ONLY
值: classes.guest_molecule(23a) | classes.guest_molecule(29) | classes.guest_molecule(5a)
---

Information of 'guest_molecule(23a)':
  - Another guest molecule encapsulated by the coordination complex.
Source of 'guest_molecule(23a)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'guest_molecule(29)':
  - A guest molecule that forms a complex with [28a·Pd]2+.
Source of 'guest_molecule(29)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'guest_molecule(5a)':
  - A guest molecule that is encapsulated by the coordination complex.
Source of 'guest_molecule(5a)':
  - https://doi.org/10.1021/acs.accounts.2c00839

Information of 'acid':
  - A category of substances including hydrochloric acid, hydrobromic acid, and sulfuric acid, used in macroc

# 研究缝隙与方案

## 近邻领域本体
1. 证据缝隙：类的某个值限制具有多个来源，是否存在证据缝隙
2. 知识缝隙：不一致的类
## 多领域本体
1. 方法论缝隙：不同的方法论带来证据的差异
2. 新路径构建（？）
3. 元科学差异：不同领域是否具有深层的“偏见”，能否通过本体结构化



In [15]:
list(acetonitrile.is_a)

[owl.Thing,
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(23a)),
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(29)),
 object_properties.is_coencapsulated_with.some(classes.guest_molecule(5a)),
 object_properties.is_hydrogen_bonded_to.some(classes.pyridyl_α-CHs),
 object_properties.is_coencapsulated_with.only(classes.acid),
 data_properties.reaction_yield_factor_23a.value('1 equiv of 23a'),
 data_properties.reaction_yield_factor_acetonitrile.value('10%'),
 object_properties.acts_as.only(owl.Nothing),
 object_properties.acts_as_stimuli_in.only(owl.Nothing),
 object_properties.binds.only(owl.Nothing),
 object_properties.binds_anion.only(owl.Nothing),
 object_properties.binds_neutral_polar_molecule.only(owl.Nothing),
 object_properties.binds_selectively.only(owl.Nothing),
 object_properties.binds_to.only(owl.Nothing),
 object_properties.binds_with.only(owl.Nothing),
 object_properties.binds_within.only(owl.Nothing),
 object_properties.can_emp

In [14]:
if Nothing in acetonitrile.equivalent_to:
    print("Yes")




Yes


In [7]:
# 遍历每个类并单独进行封闭世界操作
for cls in onto.classes():
    print(cls)
    try:
        close_world(cls)
    except Exception as e:
        print(f"Error closing world for class {cls}: {e}")

classes.oligopyrrolic_cage
classes.pyrrole_subunit
classes.anion_binding
classes.ion_channel_formation
classes.gas_absorption
classes.Cl−⊂3a−f_inclusion_complex
classes.halogen_bonding
classes.anion_supramolecular_research
classes.hydrogen_bonding
classes.catalysis
classes.ion_extraction
classes.responsive_material
classes.anion_p_bonding
classes.anion_supramolecular_chemistry
classes.anion
classes.covalent_oligopyrrolic_macrobicyclic_cage
classes.oligopyrrolic_metallacage
classes.pyrrolic_cage
classes.carbon_dioxide(CO2)
classes.oligopyrrolic_noncovalently_linked_cage
classes.oligopyrrolic_macrocycle
classes.ae-c(4)p
classes.cyclopeptide
classes.β-substituted_oligopyrrole
classes.hybrid_system
classes.heterocycle
classes.pyridine
classes.naphthyridine
classes.oligopyrrolic_polymetallic_cage
classes.oligopyrrolic_supramolecular_cage
classes.dynamic_covalent_chemistry(DCC)
classes.oligopyrrolic_cryptand
classes.cryptand
classes.oligopyrrolic_imine_cage
classes.diformyl_pyrrole_precursor